In [ ]:
# ! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [1]:
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

# MNIST Basics

Creating a basic model that can classify any image as a 3 or a 7.

## Downloading and Exploring the Data in fastai

In [6]:
# Downloading a sample of MNIST containing just 3 and 7s supplied by fastai
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [7]:
# Checking the data using the ls() method added by fastai
path.ls()

(#3) [Path('valid'),Path('labels.csv'),Path('train')]

In [8]:
# Checking the data in the training set
(path/'train').ls()

(#2) [Path('train/7'),Path('train/3')]

In [11]:
# Taking a look at each of the subfolders and using sorted() to ensure that we all get the same order of files
threes = (path/'train'/'3').ls().sorted()
sevens = (path/'train'/'7').ls().sorted()
threes

(#6131) [Path('train/3/10.png'),Path('train/3/10000.png'),Path('train/3/10011.png'),Path('train/3/10031.png'),Path('train/3/10034.png'),Path('train/3/10042.png'),Path('train/3/10052.png'),Path('train/3/1007.png'),Path('train/3/10074.png'),Path('train/3/10091.png')...]

In [ ]:
# As expected, each subfolder contains image files of threes and sevens. Let's take a look at the first image of threes.
im3_path = threes[0]
im3 = Image.open(im3_path)
im3

## Arrays vs. Tensors

Previously, we used the Image class from the *Python Imaging Library (PIL)* for opening, manipulating and viewing images.
However, in a computer, everything is represented as a number. To view the numbers that make up this image, we have to convert it to a NumPy array or a PyTorch tensor.

In [26]:
array(im3)[3:25,5:21]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,  42, 118, 219, 166, 118, 118,   6,   0,   0,   0],
       [  0,   0,   0,   0,   0, 103, 242, 254, 254, 254, 254, 254,  66,   0,   0,   0],
       [  0,   0,   0,   0,   0,  18, 232, 254, 254, 254, 254, 254, 238,  70,   0,   0],
       [  0,   0,   0,   0,   0,   0, 104, 244, 254, 224, 254, 254, 254, 141,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 207, 254, 210, 254, 254, 254,  34,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  84, 206, 254, 254, 254, 254,  41,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  24, 209, 254, 254, 254, 171,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  91, 137, 253, 254, 254, 254, 112,   0,   0],
       [  0,   0,   0,   0,   0,  40, 214, 250, 254, 254, 254, 254, 254,  34,   0,   0],
       [  0,   0,   0,   0,   0,  81, 247, 254, 254, 254, 254, 254, 254, 146,   0,   0],
       [  0,   0,   0

In [25]:
tensor(im3)[3:25,5:21]

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,  42, 118, 219, 166, 118, 118,   6,   0,   0,   0],
        [  0,   0,   0,   0,   0, 103, 242, 254, 254, 254, 254, 254,  66,   0,   0,   0],
        [  0,   0,   0,   0,   0,  18, 232, 254, 254, 254, 254, 254, 238,  70,   0,   0],
        [  0,   0,   0,   0,   0,   0, 104, 244, 254, 224, 254, 254, 254, 141,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0, 207, 254, 210, 254, 254, 254,  34,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,  84, 206, 254, 254, 254, 254,  41,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,  24, 209, 254, 254, 254, 171,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,  91, 137, 253, 254, 254, 254, 112,   0,   0],
        [  0,   0,   0,   0,   0,  40, 214, 250, 254, 254, 254, 254, 254,  34,   0,   0],
        [  0,   0,   0,   0,   0,  81, 247, 254, 254, 254, 254, 254, 254, 146,   0,   0],
        [ 

We can then use a Pandas DataFrame to colour-code the values using a gradient, which shows us clearly how the image is created from the pixel values.


In [32]:
im3_t = tensor(im3)
df = pd.DataFrame(im3_t[3:25,5:21])
df.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,42,118,219,166,118,118,6,0,0,0
2,0,0,0,0,0,103,242,254,254,254,254,254,66,0,0,0
3,0,0,0,0,0,18,232,254,254,254,254,254,238,70,0,0
4,0,0,0,0,0,0,104,244,254,224,254,254,254,141,0,0
5,0,0,0,0,0,0,0,207,254,210,254,254,254,34,0,0
6,0,0,0,0,0,0,0,84,206,254,254,254,254,41,0,0
7,0,0,0,0,0,0,0,0,24,209,254,254,254,171,0,0
8,0,0,0,0,0,0,0,91,137,253,254,254,254,112,0,0
9,0,0,0,0,0,40,214,250,254,254,254,254,254,34,0,0


We can see that the background white pixels are stored as the number 0, black is the number 255, and shades of gray are between the two.
The entire image contains 28 pixels across and 28 pixels down, for a total of 784 pixels.